In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings


d:\rag_policy\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [28]:
def load_vector_store():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")

    return FAISS.load_local(
        'faiss_index',
        embeddings,
        allow_dangerous_deserialization=True
    )


In [20]:
def retrieve_documents(vectorstore, query: str, k: int = 3):
    """Retrieve top-k similar documents."""
    results = vectorstore.similarity_search(query, k=k)
    return results

In [9]:
def main():

    query = "What is maternity leave policy?"

    # Load vector store
    vectorstore = load_vector_store()

    # Retrieve documents
    results = retrieve_documents(vectorstore, query)

    # Print results
    for i, doc in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(doc.page_content)
        print("-" * 80)


In [10]:
if __name__ == "__main__":
    main()


Result 1:
Maternity Leave Policy
Who is eligible for Maternity Leave? All female employees of the organization are eligible for maternity
leave, regardless of their tenure.
How much Maternity Leave is provided? Female employees are entitled to 26 weeks of paid maternity leave.
This leave can be taken before or after delivery based on the employee's preference and medical advice.
--------------------------------------------------------------------------------

Result 2:
Are there additional benefits with Maternity Leave? Yes, the organization may provide additional paid leave
beyond the statutory requirement as per company policy. Employees should consult HR for details on extended
maternity benefits.
Is job security guaranteed during Maternity Leave? Absolutely. No employee can be terminated during
maternity leave. The organization complies fully with the Maternity Benefit Act, 1961, which protects
employees' jobs during this period.
Paternity Leave Policy
----------------------------

In [14]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [39]:
def generate_answer(query: str):
    # Load vector store
    vectorstore = load_vector_store()


    # Retrieve documents
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join(doc.page_content for doc in docs)

    # LLM
    llm = Ollama(model="phi3", temperature=0)


    # Prompt
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an HR policy assistant.
Answer the question using ONLY the context below.

Context:
{context}

Question:
{question}

Answer:
"""
    )

    # LCEL chain
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({
        "context": context,
        "question": query
    })

    return response

In [43]:

def main():
    query = "You: Are LGBTQ+ employees protected under company policy?"
    answer = generate_answer(query)
    print("\nFinal Answer:\n")
    print(answer)




In [44]:
if __name__ == "__main__":
    main()


Final Answer:

Yes, the organization explicitly protects LGBTQ+ employees from discrimination and harassment as per their policies. They respect sexual orientation and gender identity and strive to create an inclusive environment for all employees. Additionally, support resources like employee resource groups may be available specifically for LGBTQ+ employees.
